In [1]:
from FSMSIM import *
from FSMSIM.expr import *

```
module HalfAdder {
    input a, b;
    output sum = "0";
    output carry_out = "0";

    initial state S0 {
        <- a == "0" && b == "0";
    }
    state S1 {
        <- a == "1" && b == "0";
        <- a == "0" && b == "1";
        sum = "1";
    }
    state S2 {
        <- a == "1" && b == "1";
        carry_out = "1";
    }
}
```

In [2]:
HalfAdder = Module()
# inputs
HalfAdder.define_input("a", 1)
HalfAdder.define_input("b", 1)
# state names
HalfAdder.define_state("S0")
HalfAdder.define_initial_state("S0")
HalfAdder.define_state("S1")
HalfAdder.define_state("S2")
# outputs
HalfAdder.define_output("sum", 1, StringExpr("0"))
HalfAdder.define_output("carry_out", 1, StringExpr("0"))
# S0
HalfAdder.default_state.define_transition(AndExpr(
    EqExpr(HalfAdder.inputs["a"], StringExpr("0")),
    EqExpr(HalfAdder.inputs["b"], StringExpr("0"))
), "S0")
# S1
HalfAdder.default_state.define_transition(AndExpr(
    EqExpr(HalfAdder.inputs["a"], StringExpr("1")),
    EqExpr(HalfAdder.inputs["b"], StringExpr("0"))
), "S1")
HalfAdder.default_state.define_transition(AndExpr(
    EqExpr(HalfAdder.inputs["a"], StringExpr("0")),
    EqExpr(HalfAdder.inputs["b"], StringExpr("1"))
), "S1")
HalfAdder.states["S1"].define_output("sum", StringExpr("1"))
# S2
HalfAdder.default_state.define_transition(AndExpr(
    EqExpr(HalfAdder.inputs["a"], StringExpr("1")),
    EqExpr(HalfAdder.inputs["b"], StringExpr("1"))
), "S2")
HalfAdder.states["S2"].define_output("carry_out", StringExpr("1"))

In [3]:
ha = HalfAdder.init()
ha.inputs["a"].set("1")
ha.tick()
ha.outputs["sum"], ha.outputs["carry_out"] # using __repl__

(1, 0)

In [4]:
ha.inputs["b"].set("1")
ha.tick()
ha.outputs["sum"].evaluate(), ha.outputs["carry_out"].evaluate()

('0', '1')

In [5]:
ha.inputs["a"].set("0")
ha.inputs["b"].set("0")
ha.tick()
ha.outputs["sum"].evaluate(), ha.outputs["carry_out"].evaluate()

('0', '0')

```
module OrGate {
    input a, b;
    output out = "0";
    
    initial state S0 {
        <-;
    }
    state S1 {
        <- a == "1" || b == "1";
        out = "1";
    }
}
```

In [6]:
OrGate = Module()
# inputs
OrGate.define_input("a", 1)
OrGate.define_input("b", 1)
# state names
OrGate.define_state("S0")
OrGate.define_initial_state("S0")
OrGate.define_state("S1")
# outputs
OrGate.define_output("out", 1, StringExpr("0"))
# S0
OrGate.default_state.define_transition(BoolExpr(True), "S0") # This rule will be shadowed by S1's rule
# S1
OrGate.default_state.define_transition(OrExpr(
    EqExpr(OrGate.inputs["a"], StringExpr("1")),
    EqExpr(OrGate.inputs["b"], StringExpr("1"))
), "S1")
OrGate.states["S1"].define_output("out", StringExpr("1"))

```
module FullAdder(3) {
    input a, b, carry_in;
    output sum, carry_out;

    HalfAdder ha[2];
    OrGate or;

    ha[0].a = a;
    ha[0].b = b;
    ha[1].a = carry_in;
    ha[1].b = ha[0].sum;
    or.a = ha[0].carry_out;
    or.b = ha[1].carry_out;

    sum = ha[1].sum;
    carry_out = or.out;
}
```

In [7]:
FullAdder = Module()
FullAdder.define_clock_cycles(3)
# inputs
FullAdder.define_input("a", 1)
FullAdder.define_input("b", 1)
FullAdder.define_input("carry_in", 1)
# children
FullAdder.children["ha"] = [HalfAdder.init(), HalfAdder.init()]
FullAdder.children["or"] = OrGate.init()
# wiring up children
FullAdder.children["ha"][0].inputs["a"].set(FullAdder.inputs["a"])
FullAdder.children["ha"][0].inputs["b"].set(FullAdder.inputs["b"])
FullAdder.children["ha"][1].inputs["a"].set(FullAdder.inputs["carry_in"])
FullAdder.children["ha"][1].inputs["b"].set(FullAdder.children["ha"][0].outputs["sum"])
FullAdder.children["or"].inputs["a"].set(FullAdder.children["ha"][0].outputs["carry_out"])
FullAdder.children["or"].inputs["b"].set(FullAdder.children["ha"][1].outputs["carry_out"])
# outputs
FullAdder.define_output("sum", 1, FullAdder.children["ha"][1].outputs["sum"])
FullAdder.define_output("carry_out", 1, FullAdder.children["or"].outputs["out"])

TypeError: object of type 'OutputExpr' has no len()

In [ ]:
fa = FullAdder.init()
fa.outputs["sum"].evaluate(), fa.outputs["carry_out"].evaluate()

In [ ]:
fa.inputs["a"].set("1")
fa.tick()
fa.outputs["sum"].evaluate(), fa.outputs["carry_out"].evaluate()

In [ ]:
fa.inputs["b"].set("1")
fa.tick()
fa.outputs["sum"].evaluate(), fa.outputs["carry_out"].evaluate()

In [ ]:
fa.inputs["carry_in"].set("1")
fa.tick()
fa.outputs["sum"].evaluate(), fa.outputs["carry_out"].evaluate()